In [1]:
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import util

In [2]:
tf.debugging.set_log_device_placement(True)
# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

print(tf.test.is_built_with_cuda())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)
True
Num GPUs Available:  1


In [3]:
def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel('Epochs')
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [4]:
folder_path = 'datasets/kaggle_clement/split_files/'
train_file = 'train.csv'
test_file = 'test.csv'
save_file = 'test_pred.csv'

train=pd.read_csv(folder_path+train_file, header=0)
train_X,train_Y=(np.array(train["text"]),np.array(train["label"]))

test=pd.read_csv(folder_path+test_file, header=0)
test_X,test_Y=(np.array(train["text"]),np.array(train["label"]))

In [5]:
encoder = tf.keras.preprocessing.text.Tokenizer()
encoder.fit_on_texts(train_X)
encoded_docs = encoder.texts_to_matrix(train_X, mode='tfidf')

In [6]:
print(encoded_docs)

[[0.         2.75423922 2.37664412 ... 0.         0.         0.        ]
 [0.         2.82883056 2.66842956 ... 0.         0.         0.        ]
 [0.         2.67085319 2.56544969 ... 0.         0.         0.        ]
 ...
 [0.         3.33703457 3.16724008 ... 0.         9.66237095 9.66237095]
 [0.         3.01457475 2.56544969 ... 0.         0.         0.        ]
 [0.         2.71377318 2.61878012 ... 0.         0.         0.        ]]


In [7]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=len(encoded_docs),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:GPU:

In [ ]:
history=model.fit(train_X,train_Y,epochs=10,batch_size=32)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))